# Using Open-source HuggingFace Models to Generate QAs from Raw Data

In this example, we will show you how to generate question-answers (QAs) from give text strings using open-source Huggingface models via uniflow's [HuggingFaceModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/model_flow.py#L86).

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

[In this section, we present three scenarios for illustration](#number-qas):

1. A single question and its corresponding answer.
2. A set of three questions, each with its own answer.
3. A group of five questions, again each with a specific answer.

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install libraries

In [2]:
!{sys.executable} -m pip install -q transformers accelerate bitsandbytes scipy

### Import dependency

In [3]:
from dotenv import load_dotenv
from IPython.display import display

from uniflow.flow.client import TransformClient
from uniflow.flow.config import TransformHuggingFaceConfig, HuggingfaceModelConfig
from uniflow.op.prompt import PromptTemplate, Context

load_dotenv()

False

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM, those include instruction and sample json format. We do this by giving a sample instruction and list of `Context` examples to the `PromptTemplate` class.

Here is the function `generate_QA_text`:

**Function: generate_QA_text**

**Description:**
Generates question-and-answer (QA) texts based on a given `QA_set` object and the specified number of QA pairs.

**Parameters:**
- *QA_set*: An object representing the QA set. No limitation for the number of QA sets for each context. The data structure can be found [below](#ds-example)
- *number_QAs*: The number of QA pairs to generate, contained in the question and answer sentence(s) required for the `Context` object.

**Output:**
Returns a tuple containing the context, question text, and answer text in the specified format achieving the optimal performance.

In [4]:
def generate_QA_text(QA_set, number_QAs):
    context = QA_set["Context"]
    QAs = QA_set["QAs"]

    # Initialize question and answer texts
    question_text = ""
    answer_text = ""

    for i in range(number_QAs):
        qa = QAs[i % len(QAs)]
        question_text += f"Question {i+1}: {qa['Question']} "
        answer_text += f"Answer {i+1}: {qa['Answer']} "

    # Remove trailing spaces
    question_text = question_text.strip()
    answer_text = answer_text.strip()

    return context, question_text, answer_text

<a id="number-qas"></a>
To tailor the experience to your specific needs, adjust the `number_QAs` parameter in the following code cell. This parameter allows you to define the number of question-and-answer (Q&A) pairs you wish to work with. The available options for number_QAs are 1, 3, or 5, based on our testing environment. 

If increasing `number_QAs`, please ensure your system has sufficient GPU RAM, as our examples were running on a 24 GB GPU.

In [5]:
# Modify the number of Q&A sets as desired
number_QAs = 5  # Set the value to 1, 3, or 5 as needed

Then, you can define the deatils of `sample_examples`.

If increasing the complexity of `sample_examples`, please ensure your system has sufficient GPU RAM, as our examples were running on a 24 GB GPU.
<a id="ds-example"></a>

In [6]:
from pprint import pprint

sample_instruction = """Generate {} question(s) and the corresponding answer(s) based on the context. Following \
the format of the examples below to include context, question, and answer in the response.""".format(number_QAs)

# Define the QA sets
QA_set1 = {
    "Context": "The quick brown fox jumps over the lazy dog.",
    "QAs": [
        {"Question": "What is the color of the fox?;", "Answer": "Brown;"},
        {"Question": "What is the color of the dog?;", "Answer": "Black;"},
        {"Question": "What does the fox jump over?;", "Answer": "Dog;"},
    ],
}

QA_set2 = {
    "Context": "Snoopy can be selfish, but loves his owner, Charlie Brown.",
    "QAs": [
        {"Question": "How does Snoopy sometimes behave?;", "Answer": "Selfish;"},
        {"Question": "How does Snoopy feel about his owner?;", "Answer": "Loves;"},
        {
            "Question": "What is the name of Snoopy's owner?;",
            "Answer": "Charlie Brown;",
        },
    ],
}

QA_set1_C, QA_set1_Q, QA_set1_A = generate_QA_text(QA_set1, number_QAs)
QA_set2_C, QA_set2_Q, QA_set2_A = generate_QA_text(QA_set2, number_QAs)

sample_examples = [
        Context(
            context=QA_set1_C,
            question=QA_set1_Q,
            answer=QA_set1_A
        ),
        Context(
            context=QA_set2_C,
            question=QA_set2_Q,
            answer=QA_set2_A
        )
]

guided_prompt = PromptTemplate(
    instruction=sample_instruction,
    few_shot_prompt=sample_examples
)

print("Sample_instruction:")
print(sample_instruction, '\n')
print("Sample_examples:")
pprint(sample_examples)

Sample_instruction:
Generate 5 question(s) and the corresponding answer(s) based on the context. Following the format of the examples below to include context, question, and answer in the response. 

Sample_examples:
[Context(context='The quick brown fox jumps over the lazy dog.', question='Question 1: What is the color of the fox?; Question 2: What is the color of the dog?; Question 3: What does the fox jump over?; Question 4: What is the color of the fox?; Question 5: What is the color of the dog?;', answer='Answer 1: Brown; Answer 2: Black; Answer 3: Dog; Answer 4: Brown; Answer 5: Black;'),
 Context(context='Snoopy can be selfish, but loves his owner, Charlie Brown.', question="Question 1: How does Snoopy sometimes behave?; Question 2: How does Snoopy feel about his owner?; Question 3: What is the name of Snoopy's owner?; Question 4: How does Snoopy sometimes behave?; Question 5: How does Snoopy feel about his owner?;", answer='Answer 1: Selfish; Answer 2: Loves; Answer 3: Charlie 

Second, we craft some dummy sample raw text strings. Below, we build a dataset with 400 text strings.

If increasing the complexity of `raw_context_input`, please ensure your system has sufficient GPU RAM, as our examples were running on a 24 GB GPU.

In [7]:
raw_context_input = [
    """We believe our success depends upon our capabilities in areas such as design, research and development, \
production and marketing and is supported and protected by our intellectual property rights, such as \
trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy \
of applying for and registering intellectual property rights in the United States and select foreign countries \
on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously \
protect our intellectual property, including trademarks, patents and trade secrets against third-party \
infringement and misappropriation.""",
    """In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) \
establishing the theory of information. In his article, Shannon introduced the concept of information entropy \
for the first time. We will begin our journey here.""",
    """The chain rule states that the derivative of a composite function (a function composed of another \
function) is equal to the derivative of the outer function multiplied by the derivative of the inner function.\
Mathematically, it can be written as: \(\frac{d}{dx}g(h(x)) = \frac{dg}{dh}(h(x))\cdot \frac{dh}{dx}(x)\).""",
    """Hypothesis testing involves making a claim about a population parameter based on sample data, and then \
conducting a test to determine whether this claim is supported or rejected. This typically involves \
calculating a test statistic, determining a significance level, and comparing the calculated value to a \
critical value to obtain a p-value. """
]

raw_context_input_400 = raw_context_input * 100

print("sample size of raw context: ", len(raw_context_input_400))

sample size of raw context:  400


Next, for the given raw text strings `raw_context_input` above, we convert them to the `Context` class to be processed by `uniflow`.

In [8]:

input_data = [
    Context(context=data)
    for data in raw_context_input_400
]

print("sample size of processed input data: ", len(input_data))

input_data[:2]


sample size of processed input data:  400


[Context(context='We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.'),
 Context(context='In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) establishing the theory of information. In his article, Shannon introduced the concept of information entropy for the first time. We will begin our journey here.')]

### Use LLM to generate data

In this example, we will use the [HuggingfaceModelConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/model/config.py#L39)'s default LLM to generate questions and answers. Let's import the config and client of this model.

Here, we pass in our `guided_prompt` to the `HuggingfaceConfig` to use our customized instructions and examples, instead of the `uniflow` default ones.

Note, base on your GPU memory, you can set your optimal `batch_size` below. (We attached our `batch_size` benchmarking results in the appendix of this notebook.)

In [9]:
current_batch_size = 128 if number_QAs == 1 else 64
print("batch size:", current_batch_size)

config = TransformHuggingFaceConfig(
    prompt_template=guided_prompt,
    model_config=HuggingfaceModelConfig(batch_size=current_batch_size))
client = TransformClient(config)

batch size: 64


/opt/conda/envs/uniflow/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/envs/uniflow/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [10]:
output = client.run(input_data)

  0%|          | 0/7 [00:00<?, ?it/s]

/opt/conda/envs/uniflow/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


### Process the output

Let's take a look of the generated output. We need to do a little postprocessing on the raw output.

In [11]:
import re

keywords = ["context:", "question:", "answer:"]
pattern = '|'.join(map(re.escape, keywords))

o = output[0]['output'][0]['response'][0] ## we only postprocess the first output
segments = [segment for segment in re.split(pattern, o) if segment.strip()]
result = {
    "context": segments[-3],
    "question": segments[-2],
    "answer": segments[-1]
}

pprint(result, sort_dicts=False)

{'context': ' We believe our success depends upon our capabilities in areas '
            'such as design, research and development, production and '
            'marketing and is supported and protected by our intellectual '
            'property rights, such as trademarks, utility and design patents, '
            'copyrights, and trade secrets, among others. We have followed a '
            'policy of applying for and registering intellectual property '
            'rights in the United States and select foreign countries on '
            'trademarks, inventions, innovations and designs that we deem '
            'valuable. W e also continue to vigorously protect our '
            'intellectual property, including trademarks, patents and trade '
            'secrets against third-party infringement and misappropriation.\n',
 'question': ' Question 1: What are some important factors for your success? ; '
             'Question 2: What kind of intellectual property rights do you '
   

Congrats! Your question answers from the given knowledge context are generated!


---


## Appendix

We benchmarked to see the optimal `batch_size` for the `HuggingfaceModelConfig` flow. The answer is "It depends on your data token length, your GPU memory, your LLM size, etc." In the following experiment, we use a GPU with 24G memory and a quantized LLM (2G). We still use the above 400 raw data strings `raw_context_input_400`.


Here are the results:

### number_QAs = 1
- batch_size = 128
    100%|██████████| 4/4 [03:57<00:00, 53.21s/it]
- batch_size = 64
    100%|██████████| 7/7 [05:01<00:00, 39.79s/it]
- batch_size = 4
    100%|██████████| 100/100 [39:00<00:00, 23.40s/it]
- batch_size = 1
    100%|██████████| 400/400 [18:54<00:00, 2.54s/it]

<hr/>

### number_QAs = 3
- batch_size = 128
    OOM
- batch_size = 64
    100%|██████████| 7/7 [12:09<00:00, 96.75s/it]
- batch_size = 4
    100%|██████████| 100/100 [1:32:57<00:00, 55.77s/it]
- batch_size = 1
    100%|██████████| 400/400 [31:01<00:00, 4.80s/it]

<hr/>

### number_QAs = 5
- batch_size = 128
    OOM
- batch_size = 64
    100%|██████████| 7/7 [17:36<00:00, 139.58s/it]
- batch_size = 4
    100%|██████████| 100/100 [2:21:25<00:00, 84.85s/it]
- batch_size = 1
    100%|██████████| 400/400 [47:10<00:00, 7.03s/it]